In [ ]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        # 'exintro': True,
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [1]:
from gpt_index import GPTTreeIndex, SimpleDirectoryReader

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jerryliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
documents = SimpleDirectoryReader('data').load_data()
index = GPTTreeIndex(documents)

In [3]:
index.save_to_disk('index.json')

In [2]:
# try loading
new_index = GPTTreeIndex.load_from_disk('index.json')

In [5]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# try verbose=True for more detailed outputs
new_index.query("What is the name of the professional women's basketball team in New York City?")

> Starting query: What is the name of the professional women's basketball team in New York City?
>[Level 0] Selected node: [8]/[8]
>[Level 0] Node [8] Summary text: New York City is home to the headquarters of the National Football League, Major League Baseball, the National Basketball Association, the National Hockey League, and Major League Soccer. It is the only city in the US to have teams in all four major North American professional sports leagues and is home to the world's largest parade, the Macy's Thanksgiving Day Parade. It is also home to the National Invitation Tournament, the National Tennis Center, the New York City Marathon, and the Belmont Stakes. The city has focused on reducing its environmental impact and carbon footprint, with mass transit use being the highest in the US, 3,715 hybrid taxis and other clean diesel vehicles, and the hosting of Climate Week NYC. The New York City drinking water supply is extracted from the protected Catskill Mountains watershed and is 

"The New York Liberty is the professional women's basketball team in New York City."

In [ ]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# try verbose=True for more detailed outputs
new_index.query("What battles took place in New York City in the American Revolution?", verbose=True)

In [8]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# try verbose=True for more detailed outputs
new_index.query("What are the airports in New York City?", verbose=True)

> Starting query: What are the airports in New York City?
>[Level 0] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 10),where each item in the list corresponds to a summary.
---------------------
(1) New York City, often referred to as NYC, is the most populous city in the United States. With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), it is also the most densely populated major city in the United States. Located at the southern tip of New York State, the city is based in the Eastern Time Zone and is the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area, the largest metropolitan area in the world by urban landmass. New York City is a global cultural, financial, and media center with a significant influence on commerce, health care and life sciences, entertainment, research, technology, education, politics, tourism, dining, art, fashion, and sports.

'ANSWER: None of the above. None of the summaries provided discuss airports in New York City.'